In [1]:
import glob
import face_recognition
import cv2
import numpy as np

ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
import matplotlib.pyplot as plt

### Data Set Understanding 

In [ ]:
paths = glob.glob('C:\\Users\\ayush\\Downloads\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\data\\*')
# print(paths)
names = []
images = []
image_encodings = []
image_names = []
count_img = 0
for i in paths:
    images.append(face_recognition.load_image_file(i))
    image_encodings.append(face_recognition.face_encodings(images[count_img])[0])
    image_names.append(i.split('\\')[-1].split('.')[0])
    count_img+=1
    # print(image_names)
# print(image_names)
# print(images)
# print(image_encodings)


## Data encoding, Training and detecting

In [ ]:
try:
    count = 0
    cap = cv2.VideoCapture(0)
    while True:
        ret,frame = cap.read()
        if len(frame)==0:
            break
        gray = frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(gray)
        # print(face_locations)
        face_encodings = face_recognition.face_encodings(gray, face_locations)
        # print(face_encoding)
        name = 'Unknown'
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(image_encodings, face_encoding)
            
            face_distances = face_recognition.face_distance(image_encodings, face_encoding)
            # print(face_distances)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = image_names[best_match_index]
            # if(name=='Unknown'):
                # cv2.imwrite('C:\\Users\\ayush\\Downloads\\Intruder-Detection\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\intruders\\intru-{}.jpg'.format(count),frame)
                # count+=1
            if name == 'Unknown':
                intruder =  cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                plt.imsave('C:\\Users\\ayush\\Downloads\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\intruders\\intruder.png',intruder)
                count+=1
                cap.release()
                break
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        cv2.imshow("output",frame)
        
        if(cv2.waitKey(1)==ord('q')):
            break
    cap.release()
    cv2.destroyAllWindows()
except:
    print('Intruder detected')
    cap.release()
    cv2.destroyAllWindows()

Intruder detected


## Email Image selection

In [ ]:
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

subject = "Intruder on your system"
body = "Below attached is snapshot of intruder on your system"
sender_email = "minorproject0522@gmail.com"
receiver_email = "ayushgupta0722@gmail.com"
password = 'Minor123@'

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "plain"))

filename = "C:\\Users\\ayush\\Downloads\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\Intruder-Detection-and-Automatic-Email-Alerting-system-master\\intruders\\intruder.png"  # In same directory as script

# Open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)